**Solução Para o Trabalho Prático 0**

**Problema 02:**
    
1. Usamos uma XOF e um PRG para fazer um pad que consiste em 2^N palavras de 64 bits.

2. Como pedido usamos a password como seed para o PRG.

3. Para cifrar ou decifrar fazemos um XOR como no caso do One Time Pad.

_Observação: Além do que foi mencionado para o problema 1, neste problema não há autenticação._    

**Funcionamento dos Agentes:**

* Imports do que é necessário para correr os Agentes, salientando o uso da package cryptography para o SHAKE.

In [1]:
import getpass
import multiprocessing as mp
import math
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

O Receiver e o Emitter usam as mesmas funções para a comunicação entre eles. 

* A primeira é a shake que é responsável por criar uma string longa com 2^N * 8 (bytes) de tamanho para servir de pad.

In [2]:
def shake(size, password):
    digest = hashes.Hash(hashes.SHAKE256(size), default_backend())
    digest.update(password)
    return digest.finalize()

Tendo o pad maior esta função separa o pad num array de palavras (neste caso longs).

In [3]:
def split(pad):
    n = 8
    len_pad = len(pad)
    x = [pad[i:i + n] for i in range(0, len_pad, n)]
    return x

A função xor faz XOR do pad com a mensagem. É de salientar a variável word que conta em que palavra é que vamos para nos certificarmos que não repetimos a mesma palavra duas vezes.

In [4]:
def xor(pad, message, counter):
    size = len(message)
    xored = bytearray(size)
    word = counter
    position = 0
    for i in range(size):
        xored[i] = pad[word][position] ^ message[i]
        position += 1
        if position == 8:
            word += 1
            position = 0
    return xored

**Execução do Agente Emitter:**


1. Inicialmente perguntamos qual é o valor desejado para o N, este valor tem de coincidir em ambos os agentes.

2. Depois disso calculamos o número total de palavras a gerar com a password que é pedida logo a seguir. Tendo N e a password usamos a shake para gerar o pad e a split para dividir o pad em palavras. 

3. Agora, para cada mensagem inserida verificamos se ela é maior que o pad e a nossa decisão é que embora fosse possível criar tantos pads quanto necessários para acomodar a mensagem nós decidimos que seria melhor impor um limite máximo.

4. A seguir verificamos se o tamanho da mensagem faz com que seja necessário criar um pad novo, e se for, o resto por utilizar deste pad é descartado e geramos um pad novo. 

5. Depois disto é só fazer o XOR do pad com a mensagem e enviá-la, atualizando o counter tendo em conta o tamanho da mensagem.
    
_É de salientar o uso da math.ceil nestes casos que faz com que não usemos as mesmas palavras duas vezes, por exemplo se usarmos 32 bits de uma palavra não vamos voltar a usar essa palavra, mas também não vamos usar o resto dos 32 bits._

In [5]:
def execucaoemitter(password, q, n):
    counter = 0
    # n = int(input("Emitter: N?\n"))
    lim = pow(2, n)
    # password = requestPassWord()
    pad = shake(lim * 8, password)
    pad_words = split(pad)
    stop = 1
    while stop:
        msg = input("Emitter: Escreva a sua Mensagem\n").encode()
        msg_size = len(msg)
        if msg_size < int(lim*8):
            if (counter + math.ceil(msg_size / 8)) > lim:
                print('Emitter: Recalculando o Pad\n')
                # password = requestPassWord()
                pad = shake(lim * 8, password)
                pad_words = split(pad)
                counter = 0
            ciphertext = xor(pad_words, msg, counter)
            counter += math.ceil(msg_size / 8)
            if len(ciphertext) > 0:
                q.put(ciphertext)
                print('Emitter: Mensagem Enviada')
            else:
                stop = 0
        else:
            print("Emitter: Mensagem demasiado grande\n")
            stop = 0

**Execução do Agente Receiver:**

 O Receiver funciona exatamente como o Emitter, mas em vez de se inserir uma mensagem nós recebêmo-la pela socket.

In [6]:
def execucaoreceiver(password, q, n):
    counter = 0
    # n = 2
    lim = pow(2, n)
    # password = requestPassWord()
    pad = shake(lim * 8, password)
    pad_words = split(pad)
    while True:
        msg = q.get()
        msg_size = len(msg)
        if (counter + math.ceil(msg_size/8)) > lim:
            print('Receiver: Recalculando o Pad\n')
            # password = requestPassWord()
            pad = shake(lim * 8, password)
            pad_words = split(pad)
            counter = 0
        if not msg:
            print('Receiver: Nenhuma Mensagem Recebida!\n')
            break
        else:
            cleantext = xor(pad_words, msg, counter)
            counter += math.ceil(msg_size/8)
            print("Receiver: ", cleantext.decode())

In [ ]:
if __name__ == "__main__":
    q = mp.Queue()
    p = mp.Process(target=execucaoreceiver, args=(b'teste', q, 2))
    p.start()
    execucaoemitter(b'teste', q, 2)
    p.join()

**Alínea 2.C:**

Não achamos que seja necessário fazer um grande número de medições nem outros tipos de estudos à lá Performance Engineer, mas sim podemos olhar de um ponto de vista mais removido para os dois algoritmos com que estamos a trabalhar. <br>

O gargalo do segundo algoritmo é gerar o pad, enquanto que o gargalo do primeiro pode ser o facto de que por mensagem temos de gerar uma chave, o código de autenticação para ela e cifrar o criptograma para o caso do Emitter ou então no caso do Receiver que tem o trabalho adicional de autenticar a chave que foi recebida. <br>

Tendo isto podemos imaginar facilmente que num cenário em que temos um grande número de mensagens trocadas e um N grande, a situação em que o segundo algoritmo passa pelo gargalo é rara o suficiente para o segundo algoritmo ser mais rápido que o segundo. <br>

A diferença é clara o suficiente para se poder demonstrar que o segundo algoritmo é mais rápido on demand, algo que pode ser feito facilmente na apresentação.

In [5]:
def testeexecucaoemitter(password, q, n):
    counter = 0
    # n = int(input("Emitter: N?\n"))
    lim = pow(2, n)
    # password = requestPassWord()
    pad = shake(lim * 8, password)
    pad_words = split(pad)
    i = 0
    while i < 100:
        msg = b'There is nothing either good or bad, but thinking makes it so.'
        msg_size = len(msg)
        if msg_size < int(lim*8):
            if (counter + math.ceil(msg_size / 8)) > lim:
                print('Emitter: Recalculando o Pad')
                # password = requestPassWord()
                pad = shake(lim * 8, password)
                pad_words = split(pad)
                counter = 0
            ciphertext = xor(pad_words, msg, counter)
            counter += math.ceil(msg_size / 8)
            q.put(ciphertext)
            print('Emitter: Mensagem Enviada')
        else:
            print("Emitter: Mensagem demasiado grande\n")
            stop = 0
        i += 1

In [6]:
def testeexecucaoreceiver(password, q, n):
    counter = 0
    # n = 2
    lim = pow(2, n)
    # password = requestPassWord()
    pad = shake(lim * 8, password)
    pad_words = split(pad)
    i = 0
    while i < 100:
        msg = q.get()
        msg_size = len(msg)
        if (counter + math.ceil(msg_size/8)) > lim:
            print('Receiver: Recalculando o Pad')
            # password = requestPassWord()
            pad = shake(lim * 8, password)
            pad_words = split(pad)
            counter = 0
        if not msg:
            print('Receiver: Nenhuma Mensagem Recebida!\n')
            break
        else:
            cleantext = xor(pad_words, msg, counter)
            counter += math.ceil(msg_size/8)
            print("Receiver: ", cleantext.decode())
        i += 1

In [8]:
import time
def print_elapsed_time(prefix=''):
    e_time = time.time()
    if not hasattr(print_elapsed_time, 's_time'):
        print_elapsed_time.s_time = e_time
    else:
        print(f'{prefix} elapsed time: {e_time - print_elapsed_time.s_time:.2f} sec')
        print_elapsed_time.s_time = e_time
        
if __name__ == "__main__":
    print_elapsed_time()
    q = mp.Queue()
    p = mp.Process(target=testeexecucaoreceiver, args=(b'teste', q, 8))
    p.start()
    testeexecucaoemitter(b'teste', q, 8)
    p.join()
    print_elapsed_time('Tempo para o exercício 2')

Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either goo

Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either good or bad, but thinking makes it so.
Receiver:  There is nothing either goo